# 图的最短路径问题



## Dijkstra算法
    Dijkstra算法是典型的单源最短路径算法，用于计算一个节点到其他所有节点的最短路径。主要特点是以起始点为中心向外层层扩展，直到扩展到终点为止。
1. 算法思想: 
    设G=(V,E)是一个带权有向图，把图中顶点集合V分成两组，第一组为已求出最短路径的顶点集合, 用S表示，初始时S中只有一个源点，以后每求得一条最短路径 , 就将加入到集合C中，直到全部顶点都加入到C中，算法就结束了。第二组为其余未确定最短路径的顶点集合（用U表示），按最短路径长度的递增次序依次把第二组的顶点加入C中。在加入的过程中，总保持从源点v到C中各顶点的最短路径长度不大于从源点v到U中任何顶点的最短路径长度。此外，每个顶点对应一个距离，S中的顶点的距离就是从v到此顶点的最短路径长度，U中的顶点的距离，是从v到此顶点只包括C中的顶点为中间顶点的当前最短路径长度。

2. 算法步骤:     
      a.初始时，C只包含源点，即C＝{s}，s的距离为0。U包含除s外的其他顶点，即:U={其余顶点}，若s与U中顶点u有边，则<u,s>正常有权值，若u不是s的出边邻接点，则<u,s>权值为∞。    
     b.从U中选取一个距离v最小的顶点k，把k，加入C中（该选定的距离就是s到k的最短路径长度）。    
     c.以k为新考虑的中间点，修改U中各顶点的距离；若从源点s到顶点u的距离（经过顶点k）比原来距离（不经过顶点k）短，则修改顶点u的距离值，修改后的距离值的顶点k的距离加上边上的权。    
     d.重复步骤b和c直到所有顶点都包含在C中。    

In [5]:
def shortest_path_lengths(g, src):
    '''计算从src到图g中，可以到达的顶点的最短路径长度。图g必须是带权图。返回一个字典包含每个可达顶点与src及其距离。'''
    d = {} # d[v] 是从s到v的上限
    cloud = {} # map reachable v to its d[v] value
    pq = AdaptableHeapPriorityQueue( ) # vertex v will have key d[v]
    pqlocator = {} # map from vertex to its pq locator

# for each vertex v of the graph, add an entry to the priority queue, with
# the source having distance 0 and all others having infinite distance
    for v in g.vertices( ):
        if v is src:
            d[v] = 0
        else:
            d[v] = float( inf ) # syntax for positive infinity
        pqlocator[v] = pq.add(d[v], v) # save locator for future updates

    while not pq.is_empty( ):
        key, u = pq.remove_min()
        cloud[u] = key # its correct d[u] value
        del pqlocator[u] # u is no longer in pq
        for e in g.incident_edges(u): # outgoing edges (u,v)
            v = e.opposite(u)
            if v not in cloud:
                # perform relaxation step on edge (u,v)
                wgt = e.element( )
                if d[u] + wgt < d[v]: # better path to v?
                    d[v] = d[u] + wgt # update the distance
                    pq.update(pqlocator[v], d[v], v) # update the pq entry

    return cloud # only includes reachable vertices

def shortest_path_tree(g, s, d):
    tree = { }
    for v in d:
        if v is not s:
            for e in g.incident_edges(v, False): # consider INCOMING edges
                u = e.opposite(v)
                wgt = e.element( )
                if d[v] == d[u] + wgt:
                    tree[v] = e # edge e is used to reach v
    return tree


## 最小生成树
    最小生成树是一副连通加权无向图中一棵权值最小的生成树。
    在一给定的无向图 G = (V, E) 中，(u, v) 代表连接顶点 u 与顶点 v 的边（即(u,v)in E，而 w(u, v) 代表此边的权重，若存在 T 为 E 的子集且为无循环图，使得
    w(T)=sum(w(u,v) for (u,v) in T)
    的 w(T) 最小，则此 T 为 G 的最小生成树。
    最小生成树其实是最小权重生成树的简称。一个连通图可能有多个生成树。当图中的边具有权值时，总会有一个生成树的边的权值之和小于或者等于其它生成树的边的权值之和。广义上而言，对于非连通无向图来说，它的每一连通分量同样有最小生成树。以有线电视电缆的架设为例，若只能沿着街道布线，则以街道为边，而路口为顶点，其中必然有一最小生成树能使布线成本最低。
## Prim算法
    Prim算法可在加权连通图里搜索最小生成树。意即由此算法搜索到的边子集所构成的树中，不但包括了连通图里的所有顶点，且其所有边的权值之和亦为最小。该算法于1930年由捷克数学家沃伊捷赫·亚尔尼克发现；并在1957年由美国计算机科学家罗伯特·普里姆独立发现；1959年，艾兹格·迪科斯彻再次发现了该算法。因此，在某些场合，普里姆算法又被称为DJP算法、亚尔尼克算法或普里姆－亚尔尼克算法。    
    从单一顶点开始，普里姆算法按照以下步骤逐步扩大树中所含顶点的数目，直到遍及连通图的所有顶点。    
    输入：一个加权连通图，其中顶点集合为V，边集合为E；    
    初始化：Vnew = {x}，其中x为集合V中的任一节点（起始点），Enew = {}；    
    重复下列操作，直到Vnew = V：    
    在集合E中选取权值最小的边（u, v），其中u为集合Vnew中的元素，而v则是V中没有加入Vnew的顶点（如果存在有多条满足前述条件即具有相同权值的边，则可任意选取其中之一）；    
    将v加入集合Vnew中，将（u, v）加入集合Enew中；       
    输出：使用集合Vnew和Enew来描述所得到的最小生成树。    

In [ ]:
def MST_PrimJarnik(g):
    '''Compute a minimum spanning tree of weighted graph g.

    Return a list of edges that comprise the MST (in arbitrary order).
    '''
    d = { } # d[v] is bound on distance to tree
    tree = [ ] # list of edges in spanning tree
    pq = AdaptableHeapPriorityQueue( ) # d[v] maps to value (v, e=(u,v))
    pqlocator = { } # map from vertex to its pq locator

    # for each vertex v of the graph, add an entry to the priority queue, with
    # the source having distance 0 and all others having infinite distance
    for v in g.vertices( ):
        if len(d) == 0: # this is the first node
            d[v] = 0 # make it the root
        else:
            d[v] = float('inf') # positive infinity
        pqlocator[v] = pq.add(d[v], (v,None))

    while not pq.is_empty():
        key,value = pq.remove_min()
        u,edge = value # unpack tuple from pq
        del pqlocator[u] # u is no longer in pq
        if edge is not None:
            tree.append(edge) # add edge to tree
        for link in g.incident_edges(u):
            v = link.opposite(u)
        if v in pqlocator: # thus v not yet in tree
        # see if edge (u,v) better connects v to the growing tree
            wgt = link.element( )
            if wgt < d[v]: # better edge to v?
                d[v] = wgt # update the distance
                pq.update(pqlocator[v], d[v], (v, link)) # update the pq entry
    return tree


## Kruskal算法
Kruskal算法是一种用来查找最小生成树的算法，由Joseph Kruskal在1956年发表。用来解决同样问题的还有Prim算法和Boruvka算法等。三种算法都是贪心算法的应用。和Boruvka算法不同的地方是，Kruskal算法在图中存在相同权值的边时也有效。
.算法简单描述

1).记Graph中有v个顶点，e个边    
2).新建图Graphnew，Graphnew中拥有原图中相同的e个顶点，但没有边    
3).将原图Graph中所有e个边按权值从小到大排序    
4).循环：从权值最小的边开始遍历每条边 直至图Graph中所有的节点都在同一个连通分量中    
                if 这条边连接的两个节点于图Graphnew中不在同一个连通分量中    
                                         添加这条边到图Graphnew中    

def MST Kruskal(g):
    '''Compute a minimum spanning tree of a graph using Kruskal s algorithm.

    Return a list of edges that comprise the MST.

    The elements of the graph s edges are assumed to be weights.'''
    tree = [] # list of edges in spanning tree
    pq = HeapPriorityQueue( ) # entries are edges in G, with weights as key
    forest = Partition( ) # keeps track of forest clusters
    position = {} # map each node to its Partition entry

    for v in g.vertices( ):
        position[v] = forest.make_group(v)

    for e in g.edges():
    pq.add(e.element(), e) # edge¡¯s element is assumed to be its weight

    size = g.vertex count()
    while len(tree) != size-1 and not pq.is_empty():
    #tree not spanning and unprocessed edges remain
        weight,edge = pq.remove_min()
        u,v = edge.endpoints( )
        a = forest.find(position[u])
        b = forest.find(position[v])
        if a != b:
            tree.append(edge)
            forest.union(a,b)

    return tree


参考文献:
1. Goodrich M T, Tamassia R, Goldwasser M H. Data structures and algorithms in Python[M]. John Wiley & Sons Ltd, 2013.
2. 裘宗燕. 数据结构与算法: Python语言描述. 机械工业出版社, 2016.
3. https://zh.wikipedia.org/wiki/%E5%85%8B%E9%B2%81%E6%96%AF%E5%85%8B%E5%B0%94%E6%BC%94%E7%AE%97%E6%B3%95
4. https://zh.wikipedia.org/wiki/%E6%9C%80%E5%B0%8F%E7%94%9F%E6%88%90%E6%A0%91
5. https://zh.wikipedia.org/wiki/%E6%99%AE%E6%9E%97%E5%A7%86%E7%AE%97%E6%B3%95
6. https://zh.wikipedia.org/wiki/%E6%88%B4%E5%85%8B%E6%96%AF%E7%89%B9%E6%8B%89%E7%AE%97%E6%B3%95